In [ ]:
import os
import json
import requests
import pandas as pd
from plotly import graph_objs as go

In [ ]:
url_covid_daily = "https://covid19.who.int/WHO-COVID-19-global-data.csv"
path_output_root = "data/scraped/who"
path_output_csv = os.path.join(path_output_root, "who_covid_daily.csv")
path_output_json = os.path.join(path_output_root, "who_covid_daily.json")
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
can_overwrite = False

In [ ]:
if not os.path.exists(path_output_root):
	os.makedirs(path_output_root)

if not os.path.exists(path_output_csv) or can_overwrite:
	print("Downloading CSV...")
	headers = {
		"User-Agent": user_agent
	}
	response = requests.get(url_covid_daily, headers=headers)
	with open(path_output_csv, "wb") as file:
		file.write(response.content)
	print("Done.")
else:
	print("CSV already exists - set can_overwrite to True to overwrite.")

In [ ]:
df = pd.read_csv(path_output_csv)
df.head()

In [ ]:
print("Number of countries:", df["Country"].nunique())
print("Earliest date:", df["Date_reported"].min())
print("Latest date:", df["Date_reported"].max())

In [ ]:
# Check if "World" is in country list
if "World" not in df["Country"].unique():
	print("Adding World to country list...")

In [ ]:
df_slovenia = df[df["Country"] == "Slovenia"]
df_austria = df[df["Country"] == "Austria"]
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_slovenia["Date_reported"], y=df_slovenia["Cumulative_cases"], name="Cumulative cases (Slovenia)"))
fig.add_trace(go.Scatter(x=df_austria["Date_reported"], y=df_austria["Cumulative_cases"], name="Cumulative cases (Austria)"))
# Add bars for New_cases
fig.add_trace(go.Bar(x=df_slovenia["Date_reported"], y=df_slovenia["New_cases"], name="New cases (Slovenia)"))
fig.add_trace(go.Bar(x=df_austria["Date_reported"], y=df_austria["New_cases"], name="New cases" "(Austria)"))
# Add figure title
fig.update_layout(title_text="WHO COVID-19 daily data for Slovenia")
# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes titles
fig.update_yaxes(title_text="Cumulative cases")
# Add legend
fig.update_layout(showlegend=True)
fig.show()

In [ ]:
# TODO: Get World Bank data for population and merge with WHO data to get cases per 100k people